# Exploring and Clustering Neighborhoods

# (Please skip first few lines to see code for this Clustering and map, Go to Heading Q3) Explore and Cluster Neighborhoods)

## Q1) Loading and Processing dataset 

## A) Loading the required table from Wikipedia

In [2]:
#importing Required Packages

import pandas as pd

In [3]:
tables=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [4]:
len(tables)

3

In [7]:
df=tables[0]

In [8]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df.shape

(180, 3)

## B) Removing "Not assigned" Borough

In [12]:
#Removing Not Assigned Borough
df=df[df['Borough']!= "Not assigned"]

In [13]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## C) Checking for Postal Codes that have more than 1 rows

In [17]:
df.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M2M,North York,Downsview
freq,1,24,4


We see that all Postal codes have a unique values. Hence we don't need to combine the rows with same postal codes but different neighborhoods.

## D) Checking for Neighborhoods which have "Not assigned" value but has a value for Borough

In [15]:
df[df['Neighborhood']== "Not assigned"]

,Postal Code,Borough,Neighborhood


We find that no such values are present when checking for Neighborhoods which have "Not assigned" value but has a value for Borough.  
So we don't need to assign the same neighborhood name as borough name. 

## E) Checking No of rows in the dataframe

In [22]:
print("There are {} no of rows present".format(df.shape[0]))

There are 103 no of rows present


#### Hence we notice that there are 103 rows present.

## Resetting Index

In [42]:
df=df.reset_index(drop=True)

# Q2) Getting the location of Neighborhoods

In [30]:
#Importing Required Package
from geopy.geocoders import Nominatim

We create a blank list of latitude and longitude and then store the latitude and longitude of each location in the list

In [68]:
geolocater=Nominatim(user_agent="toronto_agent")
latitude=[]
longitude=[]

for i in range(0,len(df)):
   # initialize variable to None
   location= None
   

   address =df['Neighborhood'][i]+","+df["Borough"][i]
   location = geolocater.geocode(address)
   
   if(location ==None): 
     address =df["Borough"][i]
     location = geolocater.geocode(address)
   
   latitude.append(location.latitude)
   longitude.append(location.longitude)


In [71]:
#Adding the columns to the dataframe
df["Latitude"]=latitude
df["Longitude"]=longitude

In [72]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4A,North York,Victoria Village,43.732658,-79.311189
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.754326,-79.449117
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.654174,-79.380812
4,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.668499,-79.533970


# Q3) Explore and Cluster Neighborhoods

In [81]:
import requests

In [123]:
#@hidden cell
#Define FourSquare Credential

CLIENT_ID = 'EPMCCWQMSR3TBLJ3AGJNMT24RMADYUHUHQYFUNTH2YI42PGD' # your Foursquare ID
CLIENT_SECRET = '0SA5OOZIIPG4QLAABRSRL1FF4QNN1QABXNVEEV3O45XQ1UFT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

In [93]:
toronto_data=df[df['Borough'].str.contains("Toronto")]

In [95]:
address ="Toronto"
location = geolocater.geocode(address)

latitude=location.latitude
longitude=location.longitude
print(latitude,",",longitude)

43.6534817 , -79.3839347


In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
Toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
B

In [100]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [101]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654174,-79.380812,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
1,"Regent Park, Harbourfront",43.654174,-79.380812,Elgin And Winter Garden Theatres,43.653394,-79.378507,Theater
2,"Regent Park, Harbourfront",43.654174,-79.380812,Ed Mirvish Theatre,43.655102,-79.379768,Theater
3,"Regent Park, Harbourfront",43.654174,-79.380812,Indigo,43.653515,-79.380696,Bookstore
4,"Regent Park, Harbourfront",43.654174,-79.380812,LUSH,43.653557,-79.380400,Cosmetics Shop


In [102]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,97,97,97,97,97,97
"Brockton, Parkdale Village, Exhibition Place",77,77,77,77,77,77
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",77,77,77,77,77,77
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",97,97,97,97,97,97
Central Bay Street,97,97,97,97,97,97
Christie,58,58,58,58,58,58
Church and Wellesley,64,64,64,64,64,64
"Commerce Court, Victoria Hotel",97,97,97,97,97,97
Davisville,41,41,41,41,41,41


# Analyze each Neighborhoods

In [104]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.010309,0.000000,0.000000,0.010309,0.010309,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.010309,0.010309,0.000000,0.010309
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.025974,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.012987,0.012987,0.012987,0.000000,0.012987
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.025974,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.012987,0.012987,0.012987,0.000000,0.012987
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.010309,0.000000,0.000000,0.010309,0.010309,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.010309,0.010309,0.000000,0.010309
4,Central Bay Street,0.000000,0.000000,0.010309,0.000000,0.000000,0.010309,0.010309,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.010309,0.010309,0.000000,0.010309
5,Christie,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.017241,0.017241,0.000000
6,Church and Wellesley,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.010309,0.000000,0.000000,0.010309,0.010309,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.010309,0.010309,0.000000,0.010309
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.04878,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.04878,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


# Creating DataFrame of top categories

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [109]:
import numpy as np

In [110]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Hotel,Restaurant,Diner,Middle Eastern Restaurant,Café,Chinese Restaurant,Bar,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Clothing Store,Restaurant,Café,Plaza,Seafood Restaurant,Hotel,Cosmetics Shop,Theater,Diner
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Clothing Store,Restaurant,Café,Plaza,Seafood Restaurant,Hotel,Cosmetics Shop,Theater,Diner
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Clothing Store,Hotel,Restaurant,Diner,Middle Eastern Restaurant,Café,Chinese Restaurant,Bar,Seafood Restaurant
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Restaurant,Diner,Middle Eastern Restaurant,Café,Chinese Restaurant,Bar,Seafood Restaurant


# Clustering the neighborhoods

In [115]:
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [119]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [116]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 0, 0, 1, 1], dtype=int32)

In [117]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654174,-79.380812,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Diner,Middle Eastern Restaurant,Café,Chinese Restaurant,Bar,Seafood Restaurant
3,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.654174,-79.380812,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Diner,Middle Eastern Restaurant,Café,Chinese Restaurant,Bar,Seafood Restaurant
8,M5B,Downtown Toronto,"Garden District, Ryerson",43.654174,-79.380812,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Diner,Middle Eastern Restaurant,Café,Chinese Restaurant,Bar,Seafood Restaurant
14,M5C,Downtown Toronto,St. James Town,43.654174,-79.380812,0,Coffee Shop,Clothing Store,Hotel,Restaurant,Diner,Middle Eastern Restaurant,Café,Chinese Restaurant,Bar,Seafood Restaurant
18,M4E,East Toronto,The Beaches,43.671024,-79.296712,4,Beach,Japanese Restaurant,Park,Pizza Place,Pub,Bar,Breakfast Spot,Tea Room,Sandwich Place,Skating Rink


In [122]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters